## Data Analysis in Python
Jane Rigby (Astrophysics Science Division)

Vanilla python variables (lists, tuples) are not well-suited for scientific data analysis.  Several packages have developed for real number-crunching:
* NumPy provides powerful multi-dimensional arrays and fast math.  Vectorized math is fast math!
* SciPy provides algorithms for scientific analysis (FFT, linear algebra, curve-fitting, etc).
* Pandas provides more powerful data structures that keep it all organized.

In this lecture, I assume you are already a user of NumPy, at least a little.  
* [Here is the 2016 GSFC NumPy tutorial.](https://github.com/JulesKouatchou/PBC2016/blob/master/Lectures/Day_02/02_Numpy/1DayBootCamp_Numpy.ipynb)  
* [Here is the 2016 GSFC SciPy tutorial.](https://github.com/JulesKouatchou/PBC2016/blob/master/Lectures/Day_02/04_Introduction_Scipy/PBC2016_IntroductionSciPy.ipynb)

These examples will use a combination of NumPy, SciPy, and Pandas.  
*Since Pandas is new to most people, I'll dwell on it.*

In [ ]:
# NumPy in 1 minute
import numpy as np
aa = np.array((3,1,4,1,5,9))  # 1-D numpy ndarray
print(type(aa))
print(np.median(aa))
print(np.ones_like(aa))
bb = np.zeros((3, 4))  # 2-dimensional array
print(bb)
# Numpy uses vectorized math, so avoid loops!

**What is Pandas?**  Pandas is a python library for data analysis. It uses high-level data structures, and ways to manipulate them.  Pandas comes from data science, and works (almost) seamlessly with numpy and scipy.

**Why use Pandas?**  Pandas makes it easy to manage complex data, munge data into submission, and perform scientific analysis. I've found it really fun to do science in Pandas!

**Where to get started?** I personally love the book, "Python for Data Analysis: Data Wrangling with Pandas, NumPy, and IPython", by Wes McKinney, the inventor of Pandas.

In [ ]:
from IPython.display import Image, HTML
import pandas  # Lots of people import pandas as pd for brevity
import astropy.convolution
from scipy.optimize import curve_fit
%pylab inline

<img src="https://covers.oreillystatic.com/images/0636920023784/cat.gif">

**Why should you use pandas?**  Pandas makes it easy to keep track of complex datasets.  You can visualize your data, manipulate, munge out bad data, filter, fit with models, bin and group, concatenate with other datasets, and export.

Let's dive in, and meet the basic part of pandas:  **pandas.DataFrame()**
A DataFrame is like a dictionary-on-steroids.  Or like a spreadsheet with rows and columns.

In [ ]:
# Loading the Keck/Nirspec spectrum of a gravitationally lensed galaxy, 
# from Wuyts, Rigby, Gladders et al. 2014, The Astrophysical Journal,  781, 61
df1 = pandas.read_table("Sample_data/rcs0327_knotU_N3_nirspeckeck.txt", delim_whitespace=True, comment="#")
df1.head()

So, df1 is a pandas dataframe that I read with read_table().  There are several read_ tools:  read_csv(), read_sql(), read_excel() ... you get the picture.  
I wrote a tutorial on [reading astronomical data with pandas.](https://github.com/janerigby/astro-pandas-tutorials/blob/master/Get%20data%20into%20pandas.ipynb)

Let's examine df1.  It looks like a spreadsheet, with rows and columns.

In [ ]:
print(df1.keys())
print(df1.columns)

In [ ]:
df1.info()

In [ ]:
df1.describe()

In [ ]:
df1.shape

In [ ]:
df1.tail(3)

In [ ]:
df1.head(2)

In [ ]:
df1.max()

In [ ]:
df1['flam'].nlargest(3)

In [ ]:
df1.isnull().head(3)

In [ ]:
df1.isnull().sum()

In [ ]:
# pandas.DataFrame.interpolate() can replace NaNs.  Use with care!
# pandas.dropna() can drop the NaNs.  Use with care.
df1.loc[df1['flam'].isnull()]

In [ ]:
df1.ix[3]  # Get one row, access by the index.  Clunky

In [ ]:
df1['wave'].between(12793., 12799.).head()   # Boolean filter

In [ ]:
df1.loc[df1['wave'].between(12793., 12799.)]   # Using that boolean filter

In [ ]:
# Plotting shortcut:  pandas.DataFrame.plot() 
ax = df1.plot(x='wave', y='flam', color='k')
df1.plot(x='wave', y='noise', color='yellow', ax=ax, label='uncert')
plt.ylim(0,1E-15)
plt.xlim(1.15E4, 1.38E4) # Can intermix regular matplotlib commands
plt.xlabel(r"observed wavelength ($\AA$)")

In [ ]:
#It's easy to add a new column to the dataframe
df1['dispersion'] = df1['wave'].diff()
df1['dispersion'].iloc[0] = df1['dispersion'][1] # Fill in the first value
df1.head(3)

In [ ]:
df1['smooth'] = df1['flam'].rolling(window=81, center=True).median()  # Rolling functions
plt.plot(df1['wave'], df1['flam'], color='k')        # Plotting in regular matplotlib
plt.plot(df1['wave'], df1['smooth'], color='green', lw=2)  # Smoothing example
plt.ylim(0,0.1E-15)
plt.xlim(1.15E4, 1.38E4)

In [ ]:
# A pandas.Series() is a column (or a row) of a Pandas DataFrame
example_series = df1.loc[df1['wave'].between(12793., 12799.)]['wave']
print(example_series)

In [ ]:
# NumPy should work flawlessly on a Pandas Series or Dataframe
print(np.median(df1['wave']),   df1['wave'].median())

In [ ]:
# as_matrix() converts a pandas.Series or DataFrame to a numpy ndarray:
print(example_series.as_matrix())
print(type(example_series.as_matrix()))

In [ ]:
# Some astropy and scipy functions will barf at a pandas.Series.  as_matrix() is the workaround
boxcar = 21
# This should barf:
#smooth1 = astropy.convolution.convolve(df1['flam'], np.ones((boxcar,))/boxcar, boundary='extend', fill_value=np.nan)
# But this should work:
smooth1 = astropy.convolution.convolve(df1['flam'].as_matrix(), np.ones((boxcar,))/boxcar, boundary='extend', fill_value=np.nan)

In [ ]:
df1.dropna().shape

In [ ]:
# Hack time:  
# Fit the emission lines!  Measure the integrated flux of each emission line.
# Bonus points: with uncertainties!
# Hint #1: You may want to use scipy.optimize.curve_fit()
# Hint #2: Unlike Pandas, scipy doesn't like NaN.  pandas.DataFrame.interpolate() or .dropna() can help
rest_waves = np.array((4862.683, 4960.295, 5008.240))
ax = df1.plot(x='wave', y='flam', color='k')
df1.plot(x='wave', y='noise', color='yellow', ax=ax, label='uncert')
plt.ylim(0,1E-15)
plt.xlim(1.3E4, 1.36E4) 
plt.annotate("H", xy=(1.315E4,0.2E-15), xycoords='data', fontsize=16)
plt.annotate("O++", xy=(1.34E4,0.4E-15), xycoords='data', fontsize=16)
plt.annotate("O++", xy=(1.355E4,0.8E-15), xycoords='data', fontsize=16)
plt.xlabel(r"observed wavelength ($\AA$)")
##### REMOVE BELOW HERE FOR JANE'S SOLUTION